In [12]:
import numpy as np
from gensim.models import KeyedVectors

In [ ]:
import numpy as np
from gensim.models import KeyedVectors

# Загрузка предобученных эмбеддингов GloVe
def load_glove_embeddings(file_path, embedding_dim):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Замените путь на путь к вашему файлу с эмбеддингами
file_path = 'glove.6B.300d.txt'
embedding_dim = 300
embeddings_index = load_glove_embeddings(file_path, embedding_dim)

# Пример использования: извлечение вектора для слова 'example'
vector = embeddings_index.get('example')
print(vector)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Параметры
max_words = 10000  # Максимальное количество слов в словаре
maxlen = 200       # Максимальная длина отзывов

# Создание матрицы эмбеддингов
def create_embedding_matrix(word_index, embeddings_index, embedding_dim):
    num_words = min(max_words, len(word_index) + 1)
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Загрузка текстовых данных и токенизация
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)
word_index = imdb.get_word_index()
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

embedding_matrix = create_embedding_matrix(word_index, embeddings_index, embedding_dim)

# Дополнение или обрезка отзывов до maxlen слов
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Построение модели
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Оценка модели
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')
